# Training a Classifier (CIFAR-10, PyTorch Tutorial)

This notebook follows the official PyTorch tutorial and trains a simple CNN on CIFAR-10.

## 1. Load and normalize CIFAR10
We use `torchvision` to download CIFAR-10 and apply basic normalization to map images to roughly `[-1, 1]`. If you hit multiprocessing issues on Windows/Mac, set `num_workers=0`. 

In [ ]:
import torch
import torchvision
import torchvision.transforms.v2 as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


AttributeError: module 'sympy' has no attribute 'printing'

In [4]:
import sys, os
import sympy
print("SymPy version:", getattr(sympy, "__version__", "<no version attr>"))
print("SymPy file:", getattr(sympy, "__file__", "<no file attr>"))
print("CWD:", os.getcwd())
print("Primer path (donde se buscan módulos primero):", sys.path[0])
print("¿Hay algo llamado 'sympy' aquí?", [p for p in os.listdir() if p.lower().startswith("sympy")])


SymPy version: 1.14.0
SymPy file: c:\Users\juanq\Desktop\repo\.venv\Lib\site-packages\sympy\__init__.py
CWD: c:\Users\juanq\Desktop\repo\Sprint-2\L4-computer-vision
Primer path (donde se buscan módulos primero): C:\Users\juanq\AppData\Local\Programs\Python\Python312\python312.zip
¿Hay algo llamado 'sympy' aquí? []


In [ ]:



# Transform: to Tensor and normalize to roughly [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

KeyboardInterrupt: 

## 2. Visualize some training images
Let's show a few images to get a sense of the data.

In [ ]:


# Helper: unnormalize and show an image grid
def imshow(img):
    img = img / 2 + 0.5  # unnormalize (since we normalized with mean=0.5, std=0.5)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))

# Print labels
print(' '.join(f'{classes[labels[j]]:>5s}' for j in range(batch_size)))

## 3. Define a Convolutional Neural Network
We use a small CNN similar to the tutorial: two conv+pool blocks followed by three linear layers.

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dims except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net

## 4. Define a loss function and optimizer

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 5. Train the network
We train for 2 epochs and print the running loss every 2000 mini-batches.

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

## 6. Save the trained model

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)
print(f'Model saved to {PATH}')

## 7. Test the network on a few test images

In [ ]:
# display some test images
dataiter = iter(testloader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))
print('GroundTruth:', ' '.join(f'{classes[labels[j]]:>5s}' for j in range(batch_size)))

# reload the (saved) model weights into a fresh network
net2 = Net()
net2.load_state_dict(torch.load(PATH))

outputs = net2(images)
_, predicted = torch.max(outputs, 1)
print('Predicted:  ', ' '.join(f'{classes[predicted[j]]:>5s}' for j in range(batch_size)))

## 8. Evaluate overall accuracy on the test set

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

## 9. Per-class accuracy

In [ ]:
class_correct = list(0. for _ in range(10))
class_total = list(0. for _ in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net2(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        # iterate over the batch (size = batch_size)
        for i in range(min(batch_size, labels.size(0))):
            label = labels[i].item()
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    acc = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else float('nan')
    print(f'Accuracy of {classes[i]:>5s} : {acc:.2f}%')

> Based on the official PyTorch tutorial: Training a Classifier (CIFAR-10).